# Get TNS reports and classes and cross match with objects

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import random
import collections
import pickle
from collections import OrderedDict

In [2]:
import requests

URL = 'http://tns.snad.space/api/v1/circle'


def tns_cone_search(ra_deg, dec_deg, radius_arcsec):
    r = requests.get(
        URL,
        params=dict(ra=ra_deg, dec=dec_deg, radius_arcsec=radius_arcsec),
    )
    r.raise_for_status()
    return r.json()


class NotFound(RuntimeError):
    pass


class TwoOrMoreFound(RuntimeError):
    pass

def get_tns_query(ra_deg, dec_deg, radius_arcsec):
    objects = tns_cone_search(ra_deg, dec_deg, radius_arcsec)

    if len(objects) == 0:
        return 'No_TNS'
    if len(objects) > 1:
        print(TwoOrMoreFound(f'Too many objects for ra={ra_deg}, dec={dec_deg} found: {len(objects)}'))
        return 'TwoOrMoreFound'

    obj = objects[0]
    #print(obj)
    return obj

In [3]:
dataset_bank = pd.read_csv('../dataframes/dataset_bank_90k.csv', index_col='ztf_object_id')
dataset_bank = dataset_bank[['ra', 'dec']]
dataset_bank

/Users/patrickaleo/miniconda3/envs/laiss/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (7,101,107,108,109,110) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ra,dec
ztf_object_id,,
ZTF20acpdyyq,33.976088,-20.258905
ZTF18acfvwwa,35.639509,32.833982
ZTF17aaaskvj,28.015003,36.953217
ZTF18aczbnhl,85.049145,-11.013302
ZTF18actbhqt,75.082443,-11.302676
...,...,...
ZTF18adairvu,84.348727,-2.927551
ZTF20aaiwhct,64.146691,-1.656598
ZTF20aberawv,292.910293,54.078978


In [4]:
# dataset_bank = pd.read_csv('../dataframes/dataset_bank.csv', index_col='ztf_object_id')
# dataset_bank = dataset_bank[['ra', 'dec']]
# dataset_bank

In [5]:
get_tns_query(ra_deg=160.8872687, dec_deg=18.687709451612907, radius_arcsec=1.2)

{'objid': 76055,
 'name_prefix': 'SN',
 'name': '2021bfg',
 'ra': 160.887264134,
 'declination': 18.6877333276,
 'redshift': 0.058,
 'typeid': 105.0,
 'type': 'SN Iax[02cx-like]',
 'reporting_groupid': 4.0,
 'reporting_group': 'Pan-STARRS',
 'source_groupid': 4.0,
 'source_group': 'Pan-STARRS',
 'discoverydate': '2021-01-24 14:21:07.200',
 'discoverymag': 18.65,
 'discmagfilter': 26.0,
 'filter': 'w',
 'reporters': "K. C. Chambers, T. de Boer, J. Bulger, J. Fairlamb, M. Huber, C.-C. Lin, T. Lowe, E. Magnier, A. Schultz, R. J. Wainscoat, M. Willman (IfA, University of Hawaii), K. W. Smith, D. R. Young, O. McBrien, J. Gillanders, S. Srivastav, M. Fulton, S. J. Smartt, D. O'Neill, P. Clark, S. Sim (Queen's University Belfast), D. E. Wright (University of Minnesota)",
 'time_received': '2021-01-26 10:30:33',
 'internal_names': 'PS21tj, ZTF21aagqdvr, ATLAS21dpn',
 'creationdate': '2021-01-26 10:30:34',
 'lastmodified': '2021-02-27 01:06:49',
 'coord': {'ra': 160.88726413399982, 'dec': 18.68

In [6]:
%%time

TNSresult_dict, noTNSresult_dict = {}, {}
group_name_full_dict, group_name_spec_dict, YSE_report_all_dict, YSE_report_class_dict = {}, {}, {}, {}

has_tns_l, ztf_ids_l, spec_cls_l, specz_l, report_group_l = [], [], [], [], []
for idx, (obj, ra, dec) in enumerate(zip(dataset_bank.index, dataset_bank['ra'], dataset_bank['dec'])):
    try: 
        tns_query = get_tns_query(ra_deg=ra, dec_deg=dec, radius_arcsec=1.2) # returns str or dict
    except: 
        print(f"tns_query for ra={ra}, dec={dec} has error. Skip!")
        continue
        
    ztf_ids_l.append(obj)
    
    if idx % 5000 == 0: print(idx)
    
    if isinstance(tns_query, str): 
        if (tns_query == 'No_TNS'): # Cone search is empty or has object w/ no TNS reporting (i.e star)
            noTNSresult_dict[obj] = tns_query
            has_tns_l.append('No_TNS')
            spec_cls_l.append('None'), specz_l.append(int(-99)), report_group_l.append('None')
            
        if (tns_query == 'TwoOrMoreFound'): # Cone search has two or more TNS reports at this ra/dec
            has_tns_l.append('Yes_TNS_2plus')
            spec_cls_l.append('None'), specz_l.append(int(-99)), report_group_l.append('None')
                
    else: # Cone search is NOT empty
        has_tns_l.append('Yes_TNS')
        #print(tns_query)
        tns_type = tns_query.get('type')
        #print(tns_query.get('creationdate'))
        redshift = tns_query.get('redshift')
        # Find reporting group
        report_group = tns_query.get('reporting_group')
        group_name_full_dict[obj] = report_group # all obj reporting
        
        
#         # of TNS queries, compile all which have YSE reporting group (even if aren't classified)
#         if tns_query.get('reporting_group') == 'YSE':
#             YSE_report_all_dict[obj] = tns_type if tns_type is not None else 'None'
    
        if tns_type is None: # Returns 404 cone search. 
            noTNSresult_dict[obj] = 'None'
            spec_cls_l.append('None')
            specz_l.append(int(-99))
            report_group_l.append(report_group)

        #elif tns_type is ma.masked: # Has cone search, returned obj is awaiting TNS classification
        #    TNSresult_dict[obj] = 'awaiting_class'

        else: # TNS has a classification, assign it
            TNSresult_dict[obj] = tns_type 
            spec_cls_l.append(tns_type), specz_l.append(redshift), report_group_l.append(report_group)

            # Find reporting group of spec obj
            group_name_spec_dict[obj] = report_group
            
#             # Get subset of objects reported by YSE
#             if report_group == 'YSE':
#                 YSE_report_class_dict[obj] = tns_type

0
5000
10000
15000
20000
25000
30000
Too many objects for ra=18.20966532571456, dec=32.13815482993683 found: 2
35000
Too many objects for ra=17.7471574, dec=0.4336238998396311 found: 2
40000
Too many objects for ra=198.3892150551724, dec=-1.140586562069572 found: 2
Too many objects for ra=249.66427172614863, dec=33.678278841064134 found: 2
45000
Too many objects for ra=5.5969552125, dec=40.003935390625 found: 2
50000
55000
60000
Too many objects for ra=183.1058592037036, dec=7.986730170371871 found: 2
Too many objects for ra=144.49053132271416, dec=48.38839021365505 found: 2
65000
Too many objects for ra=54.87642815285206, dec=-17.96637354429619 found: 2
70000
Too many objects for ra=213.47807927500008, dec=13.188177654167816 found: 2
Too many objects for ra=32.3890812, dec=-1.137773512301834 found: 2
Too many objects for ra=103.16703219701522, dec=51.36969596848599 found: 2
Too many objects for ra=199.05702007021657, dec=-22.558384136176656 found: 2
75000
Too many objects for ra=5.597

In [7]:
dataset_bank_tns_df = pd.DataFrame(zip(ztf_ids_l, has_tns_l, spec_cls_l, specz_l, report_group_l), columns=['ztf_object_id', 'has_tns', 'tns_cls', 'spec_z', 'report_group'])
dataset_bank_tns_df = dataset_bank_tns_df.set_index('ztf_object_id')
dataset_bank_tns_df.to_csv('../dataframes/dataset_bank_90k_tns_df.csv')
dataset_bank_tns_df

,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,
ZTF20acpdyyq,No_TNS,None,-99.0,None
ZTF18acfvwwa,No_TNS,None,-99.0,None
ZTF17aaaskvj,No_TNS,None,-99.0,None
ZTF18aczbnhl,No_TNS,None,-99.0,None
ZTF18actbhqt,No_TNS,None,-99.0,None
...,...,...,...,...
ZTF18adairvu,No_TNS,None,-99.0,None
ZTF20aaiwhct,No_TNS,None,-99.0,None
ZTF20aberawv,No_TNS,None,-99.0,None


In [8]:
# Save all dicts as pickle

list_of_dicts = [TNSresult_dict, noTNSresult_dict, group_name_full_dict, group_name_spec_dict]
list_of_dicts_str = ['TNSresult_dict_90k', 'noTNSresult_dict_90k', 'group_name_full_dict_90k', 'group_name_spec_dict_90k']

for dic, dic_str in zip(list_of_dicts, list_of_dicts_str):
    with open(f'../dataframes/{dic_str}.pickle', 'wb') as handle:
        pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)